# 1. Посмотрим на данные

План:

1) узнаем число пользователей в обучающей и тестовой выборках  
2) узнаем общее число рейтингов в train/test  
3) узнаем число уникальных ID треков в обучающей и тестовой выборках  
4) проверим, нет ли в тесте ID неизвестных треков  
5) рассмотрим распределение числа треков в истории пользователей  


In [ ]:
# 1) узнаем число пользователей в обучающей и тестовой выборках
# 2) узнаем общее число рейтингов в train/test  
# 3) узнаем число уникальных ID треков в обучающей и тестовой выборках

train_ratings_cnt = 0
train_tracks = set()

with open('train', 'r') as f:
    lines = f.readlines()
    for l in lines:
        for x in l.split():
            train_tracks.add(int(x))
            train_ratings_cnt += 1

print('Train unique tracks:', len(train_tracks))
print('Train unique users:', len(lines))
print('Train ratings count:', train_ratings_cnt)

print()

test_ratings_cnt = 0
test_tracks = set()

with open('test', 'r') as f:
    lines = f.readlines()
    for l in lines:
        for x in l.strip().split():
            test_tracks.add(int(x))
            test_ratings_cnt += 1

print('Test unique tracks:', len(test_tracks))
print('Test unique users:', len(lines))
print('Test ratings count:', test_ratings_cnt)

In [ ]:
# 4) проверим, нет ли в тесте ID неизвестных треков

print('Len of train_tracks:', len(train_tracks))
print('Max track id:', max(train_tracks))
print()

print('Len of test_tracks:', len(test_tracks))
print('Max track id:', max(test_tracks))
print()

# проверим это явно, чтобы удостовериться

warn = False
for x in test_tracks:
    if x not in train_tracks:
        warn = True
        break

if warn:
    print('Detected test track that is not in train')
else:
    print('OK')

In [ ]:
# 5) рассмотрим распределение числа треков в истории пользователей

import numpy as np
import matplotlib.pyplot as plt


tracks_per_user = []

with open('train', 'r') as f:
    lines = f.readlines()
    for l in lines:
        tracks_per_user.append(len(l.strip().split()))
        

tracks_per_user = np.array(tracks_per_user)
print("(Train) Max tracks in user's history:", max(tracks_per_user))
print("(Train) Min tracks in user's history:", min(tracks_per_user))

In [ ]:
plt.subplots(1, 2, figsize=(12,6))

plt.subplot(121)
plt.title('(Train) Users w.r.t. tracks in history')
plt.xlabel('Tracks')
plt.ylabel('Users')

plt.hist(
    tracks_per_user,
    bins=20
);


plt.subplot(122)
plt.title('(Train) Users w.r.t. tracks in history (lower counts, detailed)')
plt.xlabel('Tracks')
plt.ylabel('Users')

plt.hist(
    tracks_per_user[tracks_per_user < 50],
    bins=50
);


plt.tight_layout();

In [ ]:
# 4) рассмотрим распределение числа треков в истории пользователей

tracks_per_user_test = []

with open('test', 'r') as f:
    lines = f.readlines()
    for l in lines:
        tracks_per_user_test.append(len(l.strip().split()))
        

tracks_per_user_test = np.array(tracks_per_user_test)
print("(Test) Max tracks in user's history:", max(tracks_per_user_test))
print("(Test) Min tracks in user's history:", min(tracks_per_user_test))

In [ ]:
plt.subplots(1, 2, figsize=(12,6))

plt.subplot(121)
plt.title('(Test) Users w.r.t. tracks in history')
plt.xlabel('Tracks')
plt.ylabel('Users')

plt.hist(
    tracks_per_user_test,
    bins=20
);


plt.subplot(122)
plt.title('(Test) Users w.r.t. tracks in history (lower counts, detailed)')
plt.xlabel('Tracks')
plt.ylabel('Users')

plt.hist(
    tracks_per_user_test[tracks_per_user_test < 50],
    bins=50
);


plt.tight_layout();

Выводы: 

4) рассмотрим распределение числа треков в истории пользователей 

1) уникальных пользователей около 1 млн. 200 тыс. в обучении, 300 тыс. в тесте  
2) уникальных треков около 500 тысяч обеих выборках  
3) всего около 117 млн. рейтингов в train, test  
4) причем в тестовой выборке не встретилось неизвестных из обучения треков  
5) в истории треков у каждого пользователя в обучении не меньше 7 и не больше 256 треков,
   в тесте не меньше 6 и не больше 255 треков  
6) при этом распределения числа треков в истории пользователей в обучении и тесте выглядят 
   крайне похожими

# 2. Класс для работы с данными

In [ ]:
import scipy.sparse as sp
import numpy as np
import gc
from tqdm import tqdm

class Dataset(object):
    ''' 
        Класс для обработки обучающей и тестовой выборок
        
        1) нумерует всех пользователей из train, test
        2) собирает инфо о них в user-item матрицу
        3) выделяет из train часть для валидации
        
    '''
    def __init__(self, path):
        self.path = path
        self.val_size = None
        self.K = None
        self.train_users = None
        self.test_users = None
        self.num_items = None
        self.train_matr = None
        self.val_ratings = None
        self.val_negative_items = None
                
    def get_train_matr(self):
        self.train_matr = self.load_train_positives_matr()
    
    def set_val_data(self, val_size=0.05, K=99):
        self.val_size = val_size
        self.K = K
        self.val_ratings, self.val_negative_items = self.create_val(val_size, K)
    
    
    def count_train_test_users(self):
        num_items = 0
        train_users = 0
        test_users = 0
        
        with open(self.path + 'train', 'r') as f:
            for l in tqdm(f, desc='Count train users'):
                train_users += 1
                for i in l.split():
                    if num_items < int(i):
                        num_items = int(i)
        
        with open(self.path + 'test', 'r') as f:
            for l in tqdm(f, desc='Count test users'):
                test_users += 1
                for i in l.split():
                    if num_items < int(i):
                        num_items = int(i)
        
        self.num_items = num_items + 1   # 0, 1, ..., num_items-1
        self.train_users = train_users
        self.test_users = test_users
        
        
    
    def load_train_positives_matr(self):
        mat = sp.dok_matrix(
            (self.train_users + self.test_users, self.num_items), 
            dtype=np.float16
        )
        
        with open(self.path + 'train', 'r') as f:
            u = 0
            for l in tqdm(f, desc=f'Collect train info, {self.train_users} lines'):
                for i in l.split():
                    mat[u, int(i)] = 1.0
                u += 1
                
        with open(self.path + 'test', 'r') as f:
            u = 0            
            for l in tqdm(f, desc=f'Collect test info, {self.test_users} lines'):
                for i in l.split():
                    mat[u, int(i)] = 1.0
                u += 1
                
        return mat
    

    def create_val(self, val_size, K):
        if type(val_size) is int and val_size < 30:
            percent = val_size / 100
        elif type(val_size) is float:
            percent = val_size
        else:
            percent = -1
        
        if percent != -1:
            val_size = int(self.train_users * percent)
        
        users_idxs = np.random.choice(range(self.train_users), size=val_size, replace=False)
        users_idxs = set(users_idxs)
        
        # val_pos_pairs = np.empty((val_size,2))
        # val_negative_items = np.empty((val_size, K))
        val_pos_pairs = [-1] * val_size
        val_negative_items = [-1] * val_size
        
#         pbar = tqdm(users_idxs, desc='Iterate over val users')
#         matr = self.train_matr
#         for i, u in enumerate(pbar):
#             rated_items_idxs = matr[u].nonzero()[1]
#             item = np.random.choice(rated_items_idxs)
            
#             matr[u, item] = 0.0 # "забыли" о такой паре из train
#             val_pos_pairs[i] = [u, item]
            
#             val_negative_items[i] = np.random.randint(low=0, high=self.num_items, size=K)
            
#             # для разгрузки памяти
#             del rated_items_idxs, item
#             if i % 30000 == 0:
#                 gc.collect()

        with open(self.path + 'train', 'r') as f:
            i = 0
            for u, l in enumerate(tqdm(f, desc=f'Collect val info, {val_size} objects')):
                if u in users_idxs:
                    # будем предсказывать последний прослушанный трек
                    item = int(l.split()[-1])
                    val_pos_pairs[i] = [u, item]
                    self.train_matr[u, item] = 0.0 # "забыли" о такой паре из train
                    val_negative_items[i] = list(np.random.randint(
                        low=0, high=self.num_items, size=K))
                    i += 1                           
        
        return val_pos_pairs, val_negative_items

In [ ]:
import gc

# del ds
gc.collect()

In [ ]:
ds = Dataset('./')

In [ ]:
%%time

ds.count_train_test_users()

In [ ]:
%%time

ds.get_train_matr()

In [ ]:
ds.train_users, ds.test_users, ds.num_items

In [ ]:
import sys

mem = sys.getsizeof(ds.train_matr)
gb = mem // 1024 ** 3
mb = (mem % 1024 ** 3) // 1024 ** 2
kb = (mem % 1024 ** 2) // 1024

gb, 'GB', mb, 'MB', kb, 'KB'

In [ ]:
%%time

ds.set_val_data(val_size=50000, K=99)

In [ ]:
len(ds.val_negative_items), len(ds.val_negative_items[0])

In [ ]:
ds.val_ratings[2000:2003]

In [ ]:
# проверим на коллизии
# то есть что положительный item оказался
# среди выбранных отрицательных

assert len(ds.val_ratings) == len(ds.val_negative_items)

i_errors = []
for i in range(len(ds.val_ratings)):
    if ds.val_ratings[i][1] in ds.val_negative_items[i]:
        i_errors.append(i)

print('Error happened for i:\n', ', '.join(map(str, i_errors)))

In [ ]:
# изменим плохие объекты так чтобы брать другой индекс item

for i in i_errors[::-1]:
    for j in range(len(ds.val_negative_items[i])):
        if ds.val_negative_items[i][j] == ds.val_ratings[i][1]:
            if ds.val_negative_items[i][j] + 1 < ds.num_items:
                ds.val_negative_items[i][j] += 1
            else:
                ds.val_negative_items[i][j] -= 1

# 3. Matrix Factorization

В этой секции напишем метод матричной факторизации.  
Метод заключается в разложении матрицы user-item в произведение двух меньших матриц (малоранговое приближение) $ R = PQ $, где размеры матриц следующие (d - размерность эмбеддингов пользователя и трека): 

$R (\text{train_users + test_users}, \text{num_items})$  
$P (\text{train_users + test_users}, \text{d})$  
$Q (\text{d}, \text{num_items})$

При этом оценка конкретного трека пользователем определяется как скалярное произведение их эмбеддингов со сдвигом (bias).

In [ ]:
class MFModel(object):
    """
    Модель matrix factorization 
    Обучается с помощью SGD и negative sampling
    """

    def __init__(self, num_user, num_item, embedding_dim, reg, stddev):
        """
        reg: коэффициент регуляризации
        stddev: эмбеддинги сеплируются из нормального распределения
                с таким стандартным отклонением
        """
        self.user_embedding = np.random.normal(0, stddev, (num_user, embedding_dim))
        self.item_embedding = np.random.normal(0, stddev, (num_item, embedding_dim))
        self.user_bias = np.zeros([num_user])
        self.item_bias = np.zeros([num_item])
        self.bias = 0.0
        self.reg = reg

    def predict(self, pairs):
        """
        pairs: кортеж списков (users, items) одинаковой длины
        """
        assert len(pairs[0]) == len(pairs[1])
        num_examples = len(pairs[0])
        predictions = np.empty(num_examples)
        for i in range(num_examples):
            predictions[i] = self._predict_one(pairs[0][i], pairs[1][i])
        return predictions

    def _predict_one(self, user, item):
        return (self.bias + self.user_bias[user] + self.item_bias[item] +
                np.dot(self.user_embedding[user], self.item_embedding[item]))
    
    def fit(self, positive_pairs, learning_rate, num_negatives):
        """
        positive_pairs: [n, 2], пары user-item с понравившимися треком пользователю
        num_negatives: параметр negative sampling, равный числу отрицательных объектов
        """
        
        user_item_label_matrix = self._convert_ratings_to_implicit_data(
            positive_pairs, num_negatives
        )
        np.random.shuffle(user_item_label_matrix)

        num_examples = user_item_label_matrix.shape[0]
        reg = self.reg
        lr = learning_rate
        sum_of_loss = 0.0
        
        for i in range(num_examples):
            (user, item, rating) = user_item_label_matrix[i, :]
            user_emb = self.user_embedding[user]
            item_emb = self.item_embedding[item]
            prediction = self._predict_one(user, item)

            if prediction > 0:
                one_plus_exp_minus_pred = 1.0 + np.exp(-prediction)
                sigmoid = 1.0 / one_plus_exp_minus_pred
                this_loss = (np.log(one_plus_exp_minus_pred) +
                             (1.0 - rating) * prediction)
            else:
                exp_pred = np.exp(prediction)
                sigmoid = exp_pred / (1.0 + exp_pred)
                this_loss = -rating * prediction + np.log(1.0 + exp_pred)

            grad = rating - sigmoid

            self.user_embedding[user, :] += lr * (grad * item_emb - reg * user_emb)
            self.item_embedding[item, :] += lr * (grad * user_emb - reg * item_emb)
            self.user_bias[user] += lr * (grad - reg * self.user_bias[user])
            self.item_bias[item] += lr * (grad - reg * self.item_bias[item])
            self.bias += lr * (grad - reg * self.bias)

            sum_of_loss += this_loss

        return sum_of_loss / num_examples

    
    def _convert_ratings_to_implicit_data(self, positive_pairs, num_negatives):
        """
        Делаем из списка пар датасет для бинарной классификации
        На выходе получаем массив форму [n*(1 + num_negatives), 3]
            его строки -- (user, item, label)
        К каждой паре (user, item) из positive_pairs добавляем
            num_negatives примеров отрицательного класса -- (user, item', 0)
            где item выбираются случайно
        """
        num_items = self.item_embedding.shape[0]
        num_pos_examples = positive_pairs.shape[0]
        training_matrix = np.empty([num_pos_examples * (1 + num_negatives), 3],
                               dtype=np.int32)
        index = 0
        for pos_index in range(num_pos_examples):
            u = positive_pairs[pos_index, 0]
            i = positive_pairs[pos_index, 1]

            training_matrix[index] = [u, i, 1]
            index += 1

            for _ in range(num_negatives):
                j = i
                while j == i:
                    j = np.random.randint(num_items)
                training_matrix[index] = [u, j, 0]
                index += 1
        return training_matrix

In [ ]:
import math
import heapq # --> top_N
import numpy as np
from time import time


def eval_one_rating(model, testRatings, testNegatives, idx, top_N):
    rating = testRatings[idx]
    items = testNegatives[idx]
    u = rating[0]
    gtItem = rating[1]
    items.append(gtItem)
    
    map_item_score = {}
    users = np.full(len(items), u, dtype = 'int32')
    
    # assert type(users[0]) is int and type(np.array(items, dtype='int32')[0]) is int, \
    #        f'type(users[0]) is {type(users[0])}, type(np.array(items)[0]) is {type(np.array(items)[0])}'
    #predictions = model.predict([users, np.array(items, dtype='int32')])
    
    # assert type(users[0]) is int and type(items[0]) is int, \
    #       f'type(users[0]) is {type(users[0])}, type(items[0]) is {type(items[0])}'
    predictions = model.predict([users, items])
    
    for i in range(len(items)):
        item = items[i]
        map_item_score[item] = predictions[i]
    items.pop()
    
    ranklist = heapq.nlargest(top_N, map_item_score, key=map_item_score.get)
    hr = getHitRatio(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    mrr = getMRR(ranklist, gtItem)
    return (hr, ndcg, mrr)



def evaluate_model(model, testRatings, testNegatives, top_N):
    hits, ndcgs, mrrs = [], [], []
    for idx in range(len(testRatings)):
        (hr, ndcg, mrr) = eval_one_rating(
            model, testRatings, testNegatives, idx, top_N
        )
        hits.append(hr)
        ndcgs.append(ndcg)
        mrrs.append(mrr)
    return (hits, ndcgs, mrrs)


def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0


def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0


def getMRR(ranklist, gtItem):
    for i in range(len(ranklist)):
        if ranklist[i] == gtItem:
            return 1. / (i + 1)
    return 0


def evaluate(model, test_ratings, test_negatives, top_N=100):
    """
        Вспомогательная функция-обертка
    """
    (hits, ndcgs, mrrs) = evaluate_model(
        model, test_ratings, test_negatives, top_N=top_N
    )
    hits = np.array(hits)
    ndcgs = np.array(ndcgs)
    mrrs = np.array(mrrs)
    return hits.mean(), ndcgs.mean(), mrrs.mean()

In [ ]:
%%time

train_pos_pairs = np.column_stack(ds.train_matr.nonzero())

val_ratings, val_negative_items = (ds.val_ratings, ds.val_negative_items)

print(
    'Dataset: #user=%d, #item=%d, #train_pairs=%d, #test_pairs=%d' % (
        ds.train_users + ds.test_users, ds.num_items, 
        train_pos_pairs.shape[0], len(val_ratings)
    )
)

In [ ]:
epochs = 17
batch_size = 500
n_batches = 100

embedding_dim = 6
regularization = 0.00003
stddev = 0.02
top_N = 30
learning_rate = 0.00003

In [ ]:
del model
gc.collect()

In [ ]:
model = MFModel(
    ds.train_users + ds.test_users, 
    ds.num_items,
    embedding_dim-1, 
    regularization, 
    stddev
)

In [ ]:
hist_hr, hist_ndcg, hist_mrr = [], [], []

In [ ]:
hr, ndcg, mrr = evaluate(model, val_ratings, val_negative_items, top_N=top_N)
print('Epoch %4d:\t HR=%.4f, NDCG=%.4f\t, MRR=%.4f\t'
    % (0, hr, ndcg, mrr))

hist_hr.append(hr)
hist_ndcg.append(ndcg)
hist_mrr.append(mrr)

In [ ]:
for epoch in range(epochs):
    pbar = tqdm(range(n_batches))
    
    for batch in pbar:
        pbar.set_description(f'Epoch {epoch}, Batch {batch}/{n_batches}:')
        
        idxs = np.random.choice(
            range(len(train_pos_pairs)), size=batch_size, replace=False
        )
        
        _ = model.fit(
            train_pos_pairs[idxs], 
            learning_rate=learning_rate,
            num_negatives=top_N
        )

    hr, ndcg, mrr = evaluate(model, val_ratings, val_negative_items, top_N=top_N)
    print('Epoch %4d:\t HR=%.4f, NDCG=%.4f\t, MRR=%.4f\t'
          % (epoch+1, hr, ndcg, mrr))
    
    hist_hr.append(hr)
    hist_ndcg.append(ndcg)
    hist_mrr.append(mrr)

In [ ]:
import matplotlib.pyplot as plt


plt.subplots(1, 3, figsize=(15, 5), sharey=True)

plt.subplot(131)
plt.title('Hit Rate history')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.plot(hist_hr)
plt.grid()
plt.xticks(range(len(hist_hr)))
plt.ylim(0.0, 0.6)
plt.yticks(np.linspace(0.0, 0.6, 13))

plt.subplot(132)
plt.title('NDCG history')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.plot(hist_ndcg)
plt.grid()
plt.xticks(range(len(hist_hr)))
plt.ylim(0.0, 0.6)
plt.yticks(np.linspace(0.0, 0.6, 13))


plt.subplot(133)
plt.title('MRR history')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.plot(hist_mrr)
plt.grid()
plt.xticks(range(len(hist_hr)))
plt.ylim(0.0, 0.6)
plt.yticks(np.linspace(0.0, 0.6, 13))

plt.tight_layout()

In [ ]:
pbar = tqdm(
    range(ds.train_users, ds.train_users + ds.test_users)
)

# print('Predict best tracks for test')

top_N = 100
choose_items_num = 24000
# all_items = [i for i in range(ds.num_items)]

with open('preds.txt', 'w') as f:
    buffer = ''
    for it, u in enumerate(pbar):    
        map_item_score = {}
        # users = np.full(len(all_items), u, dtype = 'int32')
        # predictions = model.predict([users, all_items])
        items_sample = np.random.choice(
            range(ds.num_items), size=choose_items_num, replace=False
        )
        users = np.full(len(items_sample), u, dtype = 'int32')
        predictions = model.predict([users, items_sample])

        for i in range(len(items_sample)):
            # item = all_items[i]
            map_item_score[items_sample[i]] = predictions[i]

        ranklist = heapq.nlargest(top_N, map_item_score, key=map_item_score.get)
        
        buffer = buffer + ' '.join(map(str, ranklist)) + '\n'
        if (it + 1) % 200 == 0:
            f.write(buffer)
            buffer = ''
        
        # del users, predictions, map_item_score, items_sample #, ranklist, line
        # gc.collect()
    
    f.write(buffer)

# 4. Matrix Factorization 2

In [ ]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, num_train_users, num_test_users, num_items, embed_dim, stddev):
        self.num_train_users = num_train_users
        self.num_test_users = num_test_users
        self.num_items = num_items
        self.embed_dim = embed_dim
        self.stddev = stddev
        self.init_embed_normal(self.stddev)
        self.mlp = nn.Linear(
            in_features=2 * self.embed_dim,
            out_features=1
        )
        self.sigmoid = nn.Sigmoid()
    
    def init_embed_normal(self, stddev):
        self.users_embeds = nn.Embedding(self.num_train_users + self.num_test_users, self.embed_dim)
        self.items_embeds = nn.Embedding(self.num_items, self.embed_dim)
        self.users_embeds.weight = nn.Parameter(
            torch.randn(
                (self.num_train_users + self.num_test_users, self.embed_dim)
            ) * stddev
        )
        self.items_embeds.weight = nn.Parameter(
            torch.randn(
                (self.num_train_users + self.num_test_users, self.embed_dim)
            ) * stddev
        )
    
    def forward(self, users_id_batch, items_id_batch):
        batch_u_emb = self.users_embeds(users_id_batch)
        batch_i_emb = self.items_embeds(items_id_batch)
        batch_emb = torch.cat([batch_u_emb, batch_i_emb], dim=1)
        batch_logits = self.mlp(batch_emb)
        batch_preds = self.sigmoid(batch_logits)
        return batch_preds

    def predict(ui_list):
        """ ui_list = [[users, items]] """
        users = ui_list[0]
        items = ui_list[1]
        u_emb = self.users_embeds(users)
        i_emb = self.items_embeds(items)
        emb = torch.cat([u_emb, i_emb], dim=1)
        logits = self.mlp(emb)
        preds = self.sigmoid(logits)
        return preds

In [ ]:
def convert_ratings_to_implicit_data(positive_pairs, num_negatives):
        """
        Делаем из списка пар датасет для бинарной классификации
        На выходе получаем массив форму [n*(1 + num_negatives), 3]
        его строки -- (user, item, label)
        К каждой паре (user, item) из positive_pairs добавляем
        num_negatives примеров отрицательного класса -- (user, item', 0)
        где item выбираются случайно
        """
        num_items = self.item_embedding.shape[0]
        num_pos_examples = positive_pairs.shape[0]
        training_matrix = np.empty([num_pos_examples * (1 + num_negatives), 3],
                               dtype=np.int32)
        index = 0
        for pos_index in range(num_pos_examples):
            u = positive_pairs[pos_index, 0]
            i = positive_pairs[pos_index, 1]

            training_matrix[index] = [u, i, 1]
            index += 1

            for _ in range(num_negatives):
                j = i
                while j == i:
                    j = np.random.randint(num_items)
                training_matrix[index] = [u, j, 0]
                index += 1
        return training_matrix

    

def train_epoch(model, epochs, n_batches, batch_size, train_pos_pairs, num_negatives, loss_fn, opt):
    hist_hr, hist_ndcg, hist_mrr = [], [], []
    
    hr, ndcg, mrr = evaluate(model, val_ratings, val_negative_items, top_N=top_N)
    print('Epoch %4d:\t HR=%.4f, NDCG=%.4f\t, MRR=%.4f\t'
        % (0, hr, ndcg, mrr))

    hist_hr.append(hr)
    hist_ndcg.append(ndcg)
    hist_mrr.append(mrr)
    
    for epoch in range(epochs):
        pbar = tqdm(range(n_batches))

        for batch in pbar:
            opt.zero_grad()
            pbar.set_description(f'Epoch {epoch}, Batch {batch}/{n_batches}:')

            idxs = np.random.choice(
                range(len(train_pos_pairs)), size=batch_size, replace=False
            )
            
            cur_ds = train_pos_pairs[idxs]
            cur_ds = convert_ratings_to_implicit_data(cur_ds)
            batch_true = torch.Tensor(cur_ds[:, 2])
            cur_ds = torch.Tensor(cur_ds)
            batch_preds = model(cur_ds)
            loss = loss_fn(batch_preds, batch_true)
            loss.backward()
            opt.step()

        hr, ndcg, mrr = evaluate(model, val_ratings, val_negative_items, top_N=top_N)
        print('Epoch %4d:\t HR=%.4f, NDCG=%.4f\t, MRR=%.4f\t'
              % (epoch+1, hr, ndcg, mrr))

        hist_hr.append(hr)
        hist_ndcg.append(ndcg)
        hist_mrr.append(mrr)
    
    return hist_hr, hist_ndcg, hist_mrr

In [ ]:
%%time

train_pos_pairs = np.column_stack(ds.train_matr.nonzero())

val_ratings, val_negative_items = (ds.val_ratings, ds.val_negative_items)

print(
    'Dataset: #user=%d, #item=%d, #train_pairs=%d, #test_pairs=%d' % (
        ds.train_users + ds.test_users, ds.num_items, 
        train_pos_pairs.shape[0], len(val_ratings)
    )
)

In [ ]:
epochs = 17
batch_size = 500
n_batches = 100

embedding_dim = 6
regularization = 0.00003
stddev = 0.02
top_N = 30
learning_rate = 0.00003

In [ ]:
model = MLP(num_train_users, num_test_users, num_items, embed_dim, stddev)

loss_fn = nn.BCELoss()  # binary cross entropy
opt = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
hist_hr, hist_ndcg, hist_mrr = train_epoch(model, epochs, n_batches, batch_size, train_pos_pairs, num_negatives, loss_fn, opt)

In [ ]:
plt.subplots(1, 3, figsize=(15, 5), sharey=True)

plt.subplot(131)
plt.title('Hit Rate history')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.plot(new_hr)
plt.grid()
plt.xticks(range(len(hist_hr)))
plt.ylim(0.0, 0.6)
plt.yticks(np.linspace(0.0, 0.6, 13))

plt.subplot(132)
plt.title('NDCG history')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.plot(new_ndcg)
plt.grid()
plt.xticks(range(len(hist_hr)))
plt.ylim(0.0, 0.6)
plt.yticks(np.linspace(0.0, 0.6, 13))


plt.subplot(133)
plt.title('MRR history')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.plot(new_mrr)
plt.grid()
plt.xticks(range(len(hist_hr)))
plt.ylim(0.0, 0.6)
plt.yticks(np.linspace(0.0, 0.6, 13))

plt.tight_layout()

In [ ]:
pbar = tqdm(
    range(ds.train_users, ds.train_users + ds.test_users)
)

# print('Predict best tracks for test')

top_N = 100
choose_items_num = 24000
# all_items = [i for i in range(ds.num_items)]

with open('preds.txt', 'w') as f:
    buffer = ''
    for it, u in enumerate(pbar):    
        map_item_score = {}
        # users = np.full(len(all_items), u, dtype = 'int32')
        # predictions = model.predict([users, all_items])
        items_sample = np.random.choice(
            range(ds.num_items), size=choose_items_num, replace=False
        )
        users = np.full(len(items_sample), u, dtype = 'int32')
        predictions = model.predict([users, items_sample])

        for i in range(len(items_sample)):
            # item = all_items[i]
            map_item_score[items_sample[i]] = predictions[i]

        ranklist = heapq.nlargest(top_N, map_item_score, key=map_item_score.get)
        
        buffer = buffer + ' '.join(map(str, ranklist)) + '\n'
        if (it + 1) % 200 == 0:
            f.write(buffer)
            buffer = ''
        
        # del users, predictions, map_item_score, items_sample #, ranklist, line
        # gc.collect()
    
    f.write(buffer)